In [2]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import sys
sys.path.append("../src")
from myproject.pipelines.preprocessing_pipeline import preprocessing_pipeline
from myproject.pipelines.clean_pipeline import cleaning_pipeline

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Classification Models")

<Experiment: artifact_location='mlflow-artifacts:/702206731938119972', creation_time=1764089993614, experiment_id='702206731938119972', last_update_time=1764089993614, lifecycle_stage='active', name='Classification Models', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:
df = pd.read_csv("/home/jakub/Pulpit/Credit original/data/raw/train.csv")

X = df.drop(columns=["Credit_Score"])
y = df["Credit_Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


/tmp/ipykernel_8340/2167139425.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/jakub/Pulpit/Credit original/data/raw/train.csv")


In [4]:
clean = cleaning_pipeline()
prep  = preprocessing_pipeline() 

tree_clf = Pipeline([
    ("clean", clean),
    ("prep", prep),
    ("tree_clf", DecisionTreeClassifier(random_state=42))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = ["accuracy", "f1_macro"]

crossv_baseline = cross_validate(
    tree_clf,
    X_train,
    y_train,
    scoring = scoring,
    cv=cv,
    n_jobs=-1,
    return_train_score=False,
)

#metryki oceny cross validacji
cv_acc_mean_base = crossv_baseline['test_accuracy'].mean()
cv_acc_std_base = crossv_baseline['test_accuracy'].std()

cv_f1_mean_base = crossv_baseline['test_f1_macro'].mean()
cv_f1_std_base = crossv_baseline['test_f1_macro'].std()

print("           CROSS VALIDACJA")
print("|B|CV|Accuracy mean: ", cv_acc_mean_base)
print("|B|CV|Accuracy std: ", cv_acc_std_base)
print("--------------------------------------")
print("|B|CV|F1_macro mean: ", cv_f1_mean_base)
print("|B|CV|Accuracy mean: ", cv_f1_std_base)

#Trening na X_test
tree_clf.fit(X_train, y_train)

y_pred_base = tree_clf.predict(X_test)

test_acc_base = accuracy_score(y_test, y_pred_base)
test_f1_base  = f1_score(y_test, y_pred_base, average="macro")

print("           TEST")
print("|B|T|Accuracy: ", test_acc_base)
print("|B|T|F1_macro: ", test_f1_base)

print(classification_report(y_test, y_pred_base))


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline

           CROSS VALIDACJA
|B|CV|Accuracy mean:  0.67635
|B|CV|Accuracy std:  0.004197953370393707
--------------------------------------
|B|CV|F1_macro mean:  0.652528262480869
|B|CV|Accuracy mean:  0.005226388906616454


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


           TEST
|B|T|Accuracy:  0.68445
|B|T|F1_macro:  0.6605894875657007
              precision    recall  f1-score   support

        Good       0.59      0.59      0.59      3566
        Poor       0.67      0.66      0.67      5799
    Standard       0.72      0.73      0.73     10635

    accuracy                           0.68     20000
   macro avg       0.66      0.66      0.66     20000
weighted avg       0.68      0.68      0.68     20000



In [5]:
# with mlflow.start_run(run_name="Decision_Tree_Baseline"):

#     #metryki z cv
#     metrics_cv = {
#         "cv_accuracy_mean_b": float(cv_acc_mean_base), 
#         "cv_accuracy_std_b": float(cv_acc_std_base), 
#         "cv_f1_mean_b": float(cv_f1_mean_base),
#         "cv_f1_std_b": float(cv_f1_std_base)
#     }
#     mlflow.log_metrics(metrics_cv)
    
#     #metryki z test
#     metrics_t= {
#         "accuracy_base": float(test_acc_base),
#         "f1_macro_base": float(test_f1_base),
#     }
#     mlflow.log_metrics(metrics_t)


In [ ]:
params_distributions = {
    "tree_clf__max_depth": [3, 5, 7, 9, 12, None],
    "tree_clf__min_samples_split": [2, 20, 50, 100, 200],
    "tree_clf__min_samples_leaf": [1, 5, 10, 20, 50],
    "tree_clf__criterion": ["gini", "entropy"],
    "tree_clf__class_weight": [None, "balanced"],
    "tree_clf__max_features": [None, "sqrt", "log2", 0.5],
    "tree_clf__max_leaf_nodes": [None, 50, 100, 200, 500]
}

random_search = RandomizedSearchCV(
    estimator=tree_clf,
    param_distributions=params_distributions,
    n_iter=30,
    scoring="f1_macro",
    n_jobs=-1,
    cv=cv,
    random_state=42,
    verbose=2,
    return_train_score=False
    
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END tree_clf__class_weight=balanced, tree_clf__criterion=gini, tree_clf__max_depth=7, tree_clf__max_features=log2, tree_clf__max_leaf_nodes=None, tree_clf__min_samples_leaf=50, tree_clf__min_samples_split=2; total time=   3.8s
[CV] END tree_clf__class_weight=balanced, tree_clf__criterion=gini, tree_clf__max_depth=7, tree_clf__max_features=log2, tree_clf__max_leaf_nodes=None, tree_clf__min_samples_leaf=50, tree_clf__min_samples_split=2; total time=   3.7s
[CV] END tree_clf__class_weight=balanced, tree_clf__criterion=gini, tree_clf__max_depth=7, tree_clf__max_features=log2, tree_clf__max_leaf_nodes=None, tree_clf__min_samples_leaf=50, tree_clf__min_samples_split=2; total time=   4.0s

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


...

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END tree_clf__class_weight=balanced, tree_clf__criterion=entropy, tree_clf__max_depth=3, tree_clf__max_features=log2, tree_clf__max_leaf_nodes=None, tree_clf__min_samples_leaf=50, tree_clf__min_samples_split=200; total time=   3.4s


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END tree_clf__class_weight=balanced, tree_clf__criterion=entropy, tree_clf__max_depth=3, tree_clf__max_features=log2, tree_clf__max_leaf_nodes=None, tree_clf__min_samples_leaf=50, tree_clf__min_samples_split=200; total time=   2.5s


,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'tree_clf__class_weight': [None, 'balanced'], 'tree_clf__criterion': ['gini', 'entropy'], 'tree_clf__max_depth': [3, 5, ...], 'tree_clf__max_features': [None, 'sqrt', ...], ...}"
,n_iter,30
,scoring,'f1_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [7]:
tree_clf_tuned = random_search.best_estimator_

crossv_tuned = cross_validate(
    tree_clf_tuned,
    X_train,
    y_train,
    scoring = scoring,
    cv=cv,
    n_jobs=-1,
    return_train_score=False,
)

#metryki oceny cross validacji
cv_acc_mean_tuned = crossv_tuned['test_accuracy'].mean()
cv_acc_std_tuned = crossv_tuned['test_accuracy'].std()

cv_f1_mean_tuned = crossv_tuned['test_f1_macro'].mean()
cv_f1_std_tuned = crossv_tuned['test_f1_macro'].std()

print("           CROSS VALIDACJA")
print("|T|CV|Accuracy mean: ", cv_acc_mean_tuned)
print("|T|CV|Accuracy std: ", cv_acc_std_tuned)
print("--------------------------------------")
print("|T|CV|F1_macro mean: ", cv_f1_mean_tuned)
print("|T|CV|Accuracy mean: ", cv_f1_std_tuned)


y_pred_tuned = tree_clf_tuned.predict(X_test)

test_acc_tuned = accuracy_score(y_test, y_pred_tuned)
test_f1_tuned  = f1_score(y_test, y_pred_tuned, average="macro")

print("           TEST")
print("|T|T|Accuracy: ", test_acc_tuned)
print("|T|T|F1_macro: ", test_f1_tuned)

print(classification_report(y_test, y_pred_tuned))

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline

           CROSS VALIDACJA
|T|CV|Accuracy mean:  0.7063499999999999
|T|CV|Accuracy std:  0.0016039989089772027
--------------------------------------
|T|CV|F1_macro mean:  0.6850880138778664
|T|CV|Accuracy mean:  0.001783757898368959
           TEST
|T|T|Accuracy:  0.7085
|T|T|F1_macro:  0.6851571979781405
              precision    recall  f1-score   support

        Good       0.58      0.65      0.61      3566
        Poor       0.73      0.65      0.69      5799
    Standard       0.74      0.76      0.75     10635

    accuracy                           0.71     20000
   macro avg       0.69      0.69      0.69     20000
weighted avg       0.71      0.71      0.71     20000



/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [9]:
with mlflow.start_run(run_name="Decision_Tree_Tuned"):
    metrics_cv = {
        "cv_accuracy_mean_t": float(cv_acc_mean_tuned), 
        "cv_accuracy_std_t": float(cv_acc_std_tuned), 
        "cv_f1_mean_t": float(cv_f1_mean_tuned),
        "cv_f1_std_t": float(cv_f1_std_tuned)
    }
    mlflow.log_metrics(metrics_cv)
    
    metrics_t = {
        "accuracy_tuned": float(test_acc_tuned),
        "f1_macro_tuned": float(test_f1_tuned),
    }
    
    mlflow.log_metrics(metrics_t)
    

🏃 View run Decision_Tree_Tuned at: http://127.0.0.1:5000/#/experiments/702206731938119972/runs/aaff1655fff940ab895166229723799c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/702206731938119972
